# XGBoost 

(eXtreme Gradient Boosting) es una implementación optimizada de gradient boosting. Aprende secuencialmente minimizando una función de pérdida con regularización L1/L2 para prevenir overfitting. Usa árboles de decisión como base learners.


In [1]:
from src.utils import get_sample_data, get_sample_data_kaggle
from src.feature_engineerings import create_pattern_features
from src.feature_engineerings import encode_categorical_features
from src.feature_engineerings import scale_features
from src.feature_engineerings import select_important_features

import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# ============================
# 1. Cargar datos de muestra
# ============================
df_sample = get_sample_data()

# ============================
# 2. Crear variables de patrones
# ============================
df = create_pattern_features(df_sample)

# (Opcional) Análisis de patrones
#pattern_analysis = df.groupby(['off_hours', 'suspicious_frequency'])['isFraud'].mean()

# ============================
# 3. Codificar variables categóricas
# ============================
df_encode, label_encoder = encode_categorical_features(df)

# ============================
# 4. Escalar datos
# ============================
X_train, X_test, y_train, y_test, scaler = scale_features(df_encode)

# ============================
# 5. Seleccionar características importantes
# ============================
feature_importance, selected_features, selector = select_important_features(X_train, y_train)
print("Características seleccionadas:", selected_features)

# Dataset final
X_train_final = X_train[selected_features]
X_test_final  = X_test[selected_features]

# ============================
# 6. Calcular peso para balancear clases
# ============================
normal_count = len(y_train[y_train == 0])
fraud_count = len(y_train[y_train == 1])
pos_weight = normal_count / fraud_count

# ============================
# 7. Configurar modelo XGBoost
# ============================
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    subsample=0.8,
    reg_alpha=0.5,
    reg_lambda=2.0,
    gamma=0,
    colsample_bytree=1.0,
    max_depth=10,
    learning_rate=0.1,
    scale_pos_weight=pos_weight,
    random_state=42
)

# ============================
# 8. Entrenar modelo
# ============================
print("Entrenando XGBoost...")
xgb_model.fit(X_train_final, y_train)

# ============================
# 9. Evaluar modelo
# ============================
y_pred = xgb_model.predict(X_test_final)
y_prob = xgb_model.predict_proba(X_test_final)[:, 1]  

# Threshold personalizado
threshold = 0.3
y_pred_custom = (y_prob > threshold).astype(int)

# Métricas
auc_score = roc_auc_score(y_test, y_prob)
print(f"Resultados XGBoost con threshold {threshold}:")
print("AUC Score:", auc_score)
print(confusion_matrix(y_test, y_pred_custom))
print(classification_report(y_test, y_pred_custom))

print("Modelo XGBoost entrenado y evaluado.")
print("Características seleccionadas:", list(xgb_model.feature_names_in_))


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Categorical features encoded:
Total features: 29
Features escaladas: 24 variables
Training set: 80000 muestras
Test set: 20000 muestras
Tasa de fraude train: 0.0014
Tasa de fraude test: 0.0014
🔥 TOP 15 FEATURES MÁS IMPORTANTES:
 1. oldbalanceOrg             0.2581
 2. type_fraud_rate           0.1511
 3. amount                    0.0889
 4. newbalanceOrig            0.0824
 5. newbalanceDest            0.0740
 6. step                      0.0712
 7. type_encoded              0.0448
 8. oldbalanceDest            0.0386
 9. hour_of_day               0.0383
10. type_PAYMENT              0.0382
11. type_TRANSFER             0.0272
12. hour_sin                  0.0229
13. type_CASH_IN              0.0194
14. type_CASH_OUT             0.0131
15. hour_cos                  0.0103
16. day_of_week               0.0103
17. off_hours                 0.0074
18. amount_frequency          0.0024
19. weekend_activity          0.0015
20. structured_amount         0.0000

✅ 20 features seleccionadas aut

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [11 13] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Resultados XGBoost con threshold 0.3:
AUC Score: 0.9664530342479472
[[19963     9]
 [    5    23]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19972
           1       0.72      0.82      0.77        28

    accuracy                           1.00     20000
   macro avg       0.86      0.91      0.88     20000
weighted avg       1.00      1.00      1.00     20000

Modelo XGBoost entrenado y evaluado.
Características seleccionadas: [np.str_('step'), np.str_('amount'), np.str_('oldbalanceOrg'), np.str_('newbalanceOrig'), np.str_('oldbalanceDest'), np.str_('hour_of_day'), np.str_('off_hours'), np.str_('day_of_week'), np.str_('weekend_activity'), np.str_('amount_frequency'), np.str_('structured_amount'), np.str_('type_encoded'), np.str_('type_CASH_IN'), np.str_('type_CASH_OUT'), np.str_('type_DEBIT'), np.str_('type_PAYMENT'), np.str_('type_TRANSFER'), np.str_('type_fraud_rate'), np.str_('hour_sin'), np.str_('hour_cos')]


In [2]:
"""import joblib
import json

# === 1. Guardar modelo entrenado ===
joblib.dump(xgb_model, 'modelo_xgboost_fraude.pkl')

# === 2. Guardar scaler usado para normalización ===
joblib.dump(scaler, 'scaler.pkl')

# === 3. Guardar selector de features importantes ===
joblib.dump(selector, 'selector_kbest.pkl')

# === 4. Guardar nombres de las features seleccionadas ===
joblib.dump(selected_features, 'features_seleccionadas.pkl')

# 🔹 Mejor guardar solo los nombres de columnas, no todo X_train
columnas_entrenamiento = list(X_train.columns)
joblib.dump(columnas_entrenamiento, 'columnas_entrenamiento.pkl')

# === 5. Guardar threshold personalizado en un archivo JSON (más flexible) ===
config = {"threshold": 0.3}
with open('config_modelo.json', 'w') as f:
    json.dump(config, f)

print("✅ Todo fue guardado exitosamente para producción.")"""


'import joblib\nimport json\n\n# === 1. Guardar modelo entrenado ===\njoblib.dump(xgb_model, \'modelo_xgboost_fraude.pkl\')\n\n# === 2. Guardar scaler usado para normalización ===\njoblib.dump(scaler, \'scaler.pkl\')\n\n# === 3. Guardar selector de features importantes ===\njoblib.dump(selector, \'selector_kbest.pkl\')\n\n# === 4. Guardar nombres de las features seleccionadas ===\njoblib.dump(selected_features, \'features_seleccionadas.pkl\')\n\n# 🔹 Mejor guardar solo los nombres de columnas, no todo X_train\ncolumnas_entrenamiento = list(X_train.columns)\njoblib.dump(columnas_entrenamiento, \'columnas_entrenamiento.pkl\')\n\n# === 5. Guardar threshold personalizado en un archivo JSON (más flexible) ===\nconfig = {"threshold": 0.3}\nwith open(\'config_modelo.json\', \'w\') as f:\n    json.dump(config, f)\n\nprint("✅ Todo fue guardado exitosamente para producción.")'

## DESPLIEGUE ANTIFRAUDE MODELO XGBOOST

In [16]:
import pandas as pd
import joblib
import json

# === 1. Cargar componentes entrenados ===
model = joblib.load('modelo_xgboost_fraude.pkl')
scaler = joblib.load('scaler.pkl')
selector = joblib.load('selector_kbest.pkl')
columnas_entrenamiento = joblib.load('columnas_entrenamiento.pkl')  # lista de nombres de columnas

# === 2. Cargar configuración del modelo (threshold, etc.) ===
with open('config_modelo.json', 'r') as f:
    config = json.load(f)
threshold = config["threshold"]

# === 2. Cargar configuración del modelo (threshold, etc.) ===
with open('config_modelo.json', 'r') as f:
    config = json.load(f)
threshold = config["threshold"]

# === 3. Diccionario de datos de prueba (ejemplo de fraude) ===
data_prueba_dict = {
        "step": 50,                 # Paso de tiempo en el dataset (unidad: horas simuladas)
        "amount": 9500.0,           # Monto muy alto comparado con el promedio del usuario
        "oldbalanceOrg": 9700.0,    # Saldo antes de la transacción
        "newbalanceOrig": 200.0,    # Saldo después de la transacción (casi vacía la cuenta)
        "oldbalanceDest": 100.0,    # Saldo previo del receptor
        "newbalanceDest": 9600.0,   # Saldo después de recibir el dinero
        "hour_of_day": 2,           # Transacción a las 2 AM
        "off_hours": 1,             # Fuera de horario laboral (indicio de riesgo)
        "day_of_week": 0,           # Lunes
        "weekend_activity": 0,      # No es fin de semana
        "amount_frequency": 0,      # Usuario no realiza este monto habitualmente
        "suspicious_frequency": 1,  # Actividad repetitiva sospechosa en poco tiempo
        "structured_amount": 0,     # No es fraccionamiento, pero sí monto elevado
        "dest_diversity": 0,        # Destinatario poco frecuente
        "high_dest_diversity": 0,   # No hay gran diversidad de destinos
        "type_encoded": 4,          # Tipo de transacción codificado
        "type_CASH_IN": 0,
        "type_CASH_OUT": 0,
        "type_DEBIT": 0,
        "type_PAYMENT": 0,
        "type_TRANSFER": 1,         # Fue una transferencia
        "type_fraud_rate": 0.25,    # Tipo de transacción con alta tasa histórica de fraude
        "hour_sin": 0.51,           # Representación trigonométrica de la hora
        "hour_cos": -0.86           # Representación trigonométrica de la hora

}

# === 4. Convertir a DataFrame y alinear columnas ===
data_prueba = pd.DataFrame([data_prueba_dict])

# Alinear columnas exactamente como en el entrenamiento
data_prueba = data_prueba[columnas_entrenamiento]

# === 5. Escalar datos ===
data_scaled = scaler.transform(data_prueba)

# === 6. Seleccionar features importantes ===
data_selected = selector.transform(data_scaled)

# === 7. Obtener predicciones ===
probabilidad = model.predict_proba(data_selected)[0][1]
prediccion = int(probabilidad > threshold)  # umbral personalizado

# === 8. Mostrar resultados ===
print("¿Es fraude?:", "✅ SÍ" if prediccion == 1 else "❌ NO")
print(f"Probabilidad de fraude: {probabilidad:.2%}")


¿Es fraude?: ✅ SÍ
Probabilidad de fraude: 13.97%


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SelectKBest was fitted with feature names
  warnings.warn(


In [21]:
# probar_modelo.py
import pandas as pd
import joblib
import json
from data_test import ejemplos  # Importar lista de ejemplos

# === 0. Elegir manualmente el caso ===
opcion = 4  # Cambia este número (0 a 4) para probar otro caso
# Cada elemento de "ejemplos" es un diccionario con una clave "Fraude" o "Legitimo"
caso_dict = list(ejemplos[opcion].values())[0]  # Extraer el diccionario de features

# === 1. Cargar componentes entrenados ===
model = joblib.load('modelo_xgboost_fraude.pkl')
scaler = joblib.load('scaler.pkl')
selector = joblib.load('selector_kbest.pkl')
columnas_entrenamiento = joblib.load('columnas_entrenamiento.pkl')  # lista de nombres de columnas

# === 2. Cargar configuración del modelo (threshold, etc.) ===
with open('config_modelo.json', 'r') as f:
    config = json.load(f)
threshold = config["threshold"]

# === 3. Convertir a DataFrame y alinear columnas ===
data_prueba = pd.DataFrame([caso_dict])
data_prueba = data_prueba[columnas_entrenamiento]  # Alinear con entrenamiento

# === 4. Escalar datos ===
data_scaled = scaler.transform(data_prueba)

# === 5. Seleccionar features importantes ===
data_selected = selector.transform(data_scaled)

# === 6. Obtener predicciones ===
probabilidad = model.predict_proba(data_selected)[0][1]
prediccion = int(probabilidad > threshold)  # umbral personalizado

# === 7. Mostrar resultados ===
print(f"Caso probado: {list(ejemplos[opcion].keys())[0]}")
print("¿Es fraude?:", "✅ SÍ" if prediccion == 1 else "❌ NO")
print(f"Probabilidad de fraude: {probabilidad:.2%}")


Caso probado: Fraude
¿Es fraude?: ✅ SÍ
Probabilidad de fraude: 2.55%


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SelectKBest was fitted with feature names
  warnings.warn(
